In [1]:
import os
import pandas as pd

### There you can change parameters, which is controlling GA

In [2]:
n_organisms = 256
n_elites = 16
n_generations = 2

config = {'n_organisms': n_organisms,
          'n_elites': n_elites,
          'n_generations': n_generations,
          
          'crossover_rate': 1.0,
          'mutation_rate': 1.0,
          'gamma': 0.05,
         }

In [3]:
output_folder_name = 'NEW_NAME'
output_folder_path = '../../results/'

config['output_folder_name'] = os.path.join(output_folder_path, output_folder_name)

#### Processing data and c-typesmodel

List of losses is contained in file of losses : "ga/mpi_scripts/loss_utils.py"

If you want to another type of loss - write new loss function and it's name in file of losses

In [4]:
config['loss'] = 'RMSE'

#### There you can change with wich columns would work GA

Data, which we want to optimize parameters(data from patch-clamp), after preprocessing is saved in column by name *"I_out"*

Cmodel create dataframe by protocol and called it by name *"I_out"*

In protocol we using column *'v'*, because our model use command potential and create current

**REMEMBER:** Its all depend on how you save trace, protocol and what created c-model

In [5]:
config['columns_control'] = ['I_out']
config['columns_model'] = ['I_out']
config['column_stim_protocol'] = ['v']

#### Choose which model you want to use , and how called your file.so

In [6]:
dirname_model = '../src/model_ctypes/PC_model/'
filename_so = 'ina.so'

config['filename_so'] =  os.path.abspath(os.path.join(dirname_model, filename_so))
config['filename_legend_states'] =  os.path.abspath(os.path.join(dirname_model, 'legend_states.csv'))
config['filename_legend_constants'] =  os.path.abspath(os.path.join(dirname_model, 'legend_constants.csv'))
config['filename_legend_algebraic'] =  os.path.abspath(os.path.join(dirname_model, 'legend_algebraic.csv'))

In [7]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/NEW_NAME',
 'loss': 'RMSE',
 'columns_control': ['I_out'],
 'columns_model': ['I_out'],
 'column_stim_protocol': ['v'],
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/ina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv'}

# Now lets choose which parameters you want to optimize and which trace

In [8]:
config['experimental_conditions'] = {}

In *common* part written all parameters of patch-clamp experiment, in *individual* part - parameters of sodium current  and paths to initial state protocol, protocol and trace.

In [9]:
default_settings = {'bounds': [0.1, 10], 'is_multiplier': True}

common = {}
common_parameters = ['a0_m','b0_m','delta_m','s_m', 'tau_m_const',
                    'a0_h', 'b0_h','delta_h','s_h', 'tau_h_const',
                    'a0_j', 'b0_j', 'delta_j', 's_j', 'tau_j_const',
                    ]


# You can use default bounds and is_multipliers
common['params'] = {param: default_settings for param in common_parameters}

# If it nesessary, you can change bounds for some parameters
common['params']['tau_m_const']['bounds'] =  [0.05, 2.0]



Below prepared parameters bounds, which we used in our paper 

In [10]:
common['params']['v_half_h'] = {'bounds': [60.0, 80.0], 'is_multiplier': False}
common['params']['v_half_m'] = {'bounds': [20.0, 30.0], 'is_multiplier': False}
common['params']['k_h'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}
common['params']['k_m'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}

The same as in *common* part, we doing with *individual* part

In [11]:
individual = {}
individual['params'] = {'c_p': {'bounds': [0.01,5.0], 'is_multiplier': True},
                   'c_m': {'bounds': [0.1, 5.0],  'is_multiplier': True},
                   'R': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'R_f': {'bounds': [0.01, 50.0], 'is_multiplier': True},
                   'g_max': {'bounds': [0.05, 100.0], 'is_multiplier': True},
                   'g_leak': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'x_c_comp': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'x_r_comp': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'alpha': {'bounds': [0.72, 0.78], 'is_multiplier': False},
                   'v_rev': {'bounds': [16.0, 80.0], 'is_multiplier': False},
                   'v_off': {'bounds': [-2.0, 2.0], 'is_multiplier': False},
                   'tau_z': {'bounds': [0.4, 6.0], 'is_multiplier': True}}

##### Don't forget to write below the dirs for traces. It will be written in *individual*

Now where written default paths

In [12]:
trace_dir = '../data/traces/'
protocol_dir = '../data/protocols/'
initial_state_protocol_dir = '../data/initial_protocols/'

#### Choose the data, which you want to use in GA
trace number - n_trace,

protocol number - n_protocol

initial state protocol - n_initial_state_protocol



In [13]:
### TRACES
traces = pd.DataFrame(os.listdir(trace_dir), columns=['filename'])
traces.index.name ='n_trace'
traces

,filename
n_trace,
0,NEW_NAME.csv
1,inactivation.csv
2,activation.csv


In [14]:
### PROTOCOLS

protocols = pd.DataFrame(os.listdir(protocol_dir), columns=['filename'])
protocols.index.name ='n_protocol'
protocols

,filename
n_protocol,
0,NEW_NAME.csv
1,inactivation.csv
2,activation.csv


In [15]:
### INITIAL STATE PROTOCOLS

initial_state_protocols = pd.DataFrame(os.listdir(initial_state_protocol_dir), columns=['filename'])
initial_state_protocols.index.name ='n_trace'
initial_state_protocols

,filename
n_trace,
0,NEW_NAME.csv
1,.~lock.activation_initial_state.csv#
2,activation_initial_state.csv
3,inactivation_initial_state.csv


In [16]:
n_trace = 0
n_protocol = 1
n_initial_state_protocol = 3


trace_path = os.path.abspath(os.path.join(trace_dir, traces.filename[n_trace]))
protocol_path = os.path.abspath(os.path.join(protocol_dir, protocols.filename[n_protocol]))
initial_state_protocol_path = os.path.abspath(os.path.join(initial_state_protocol_dir, 
                                           initial_state_protocols.filename[n_initial_state_protocol]))


individual['filename_phenotype'] = trace_path
individual['filename_protocol'] = protocol_path
individual['filename_initial_state_protocol'] = initial_state_protocol_path

In [17]:
config['experimental_conditions']['common'] = common
config['experimental_conditions']['individual'] = individual

In [18]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/NEW_NAME',
 'loss': 'RMSE',
 'columns_control': ['I_out'],
 'columns_model': ['I_out'],
 'column_stim_protocol': ['v'],
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/ina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'experimental_conditions': {'common': {'params': {'a0_m': {'bounds': [0.05,
      2.0],
     'is_multiplier': True},
    'b0_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    'delta_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    's_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
  

In [19]:
# import json

# config_name = 'test'
# with open(f"../ga/configs/{config_name}.json", 'w') as f:
#     f.write(json.dumps(config, indent=4))

## Advanced 
In GA there possibility to optimize parameters of several traces. For this you should create dictionary *individual_1*(or with another name) in the same way as was done with *individual*, and add it in config. In this case GA would found same sodium current parameters, but different parameters for each experiment     

In [20]:
import copy
advanced_config = copy.deepcopy(config)
advanced_config['experimental_conditions']['individual_1'] = individual

In [21]:
# import json

# advanced_config_name = 'advanced_config'
# with open(f"../ga/configs/{advanced_config_name}.json", 'w') as f:
#     f.write(json.dumps(advanced_config, indent=4))